# Resultados gerais

In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
from generate_tsp import GerarProblemaRadialTSP
from sga import SGAPermutacao, SGAInteiroLimitado

In [3]:
def EscreverResultado(res,params,topologia,tipo,vetorOtimo):
    params['Topologia'] = topologia
    pop,stats,hof,params,custo = res
    resPerm = pd.Series(params)
    resPerm['Tipo'] = tipo
    minVal, gap, success, optSuccess = stats.select("min",'gap','success','optimal success')
    resPerm['Sucesso fraco'] = np.any(success)
    resPerm['Sucesso'] = np.any(optSuccess)
    resPerm['Geração final fraco'] = np.argwhere(success)[0][0] if np.any(success) else len(success)
    resPerm['Geração final'] = np.argwhere(optSuccess)[0][0] if np.any(optSuccess) else len(optSuccess)
    resPerm['MBF'] = np.mean(minVal)
    resPerm['Valor encontrado'] = hof[0].fitness.values[0]
    resPerm['Gap'] = custo(vetorOtimo)[0]/resPerm['Valor encontrado']
    resPerm['Indivíduo encontrado'] = hof[0]
    return pd.DataFrame([resPerm])
    
def Experimento(posicoes,params,nomePosicao='Padrão',vetorOtimo=None):
    res = SGAPermutacao(posicoes,params,vetorOtimo=vetorOtimo)
    resPerm = EscreverResultado(res,params,nomePosicao,'Permutação',vetorOtimo)
    res = SGAInteiroLimitado(posicoes,params,vetorOtimo=vetorOtimo)
    resInt = EscreverResultado(res,params,nomePosicao,'Inteiro',vetorOtimo)
    return pd.concat([resPerm,resInt],ignore_index=True)

In [22]:
def CalcularSR(resultados):
    return (100*resultados.groupby(['Topologia','Tipo','P','mu'])['Sucesso'].sum()/resultados.groupby(['Topologia','Tipo','P','mu'])['Sucesso'].count()).unstack(['Topologia',"Tipo"])
CalcularMBF = lambda resultados: resultados.groupby(['Topologia','Tipo','P','mu'])['MBF'].describe().reset_index().set_index(['Topologia','Tipo','P','mu']).unstack(['Topologia','Tipo'])
CalcularAES = lambda resultados: resultados[resultados['Sucesso'] == True].groupby(['Topologia','Tipo','P','mu'])['Geração final'].describe().unstack(['Topologia','Tipo'])
def CalcularSRFraco(resultados):
    return (100*resultados.groupby(['Topologia','Tipo','P','mu'])['Sucesso fraco'].sum()/resultados.groupby(['Topologia','Tipo','P','mu'])['Sucesso fraco'].count()).unstack(['Topologia',"Tipo"])
CalcularAESFraco = lambda resultados: resultados[resultados['Sucesso fraco'] == True].groupby(['Topologia','Tipo','P','mu'])['Geração final fraco'].describe().unstack(['Topologia','Tipo'])

In [5]:
dbconn = sqlite3.connect("Resultados-finais.sqlite")
resultados = pd.read_sql("SELECT * FROM resultados",dbconn)
resultados

,index,N,P,mu,cxpb,mupb,muidxpb,Topologia,Tipo,Sucesso fraco,Sucesso,Geração final fraco,Geração final,MBF,Valor encontrado,Gap,Indivíduo encontrado
0,0,1000,11,20,0.7,0.2,0.05,Ordenada,Permutação,1,1,109,109,6.473602,6.180340,1.000000,b'\n\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x...
1,1,1000,11,20,0.7,0.2,0.05,Ordenada,Inteiro,1,1,40,40,6.227524,6.180340,1.000000,b'\x08\x00\x00\x00\t\x00\x00\x00\t\x00\x00\x00...
2,2,1000,11,20,0.7,0.2,0.05,Aleatória,Permutação,1,1,252,252,6.846630,6.180340,1.000000,b'\x08\x00\x00\x00\x00\x00\x00\x00\t\x00\x00\x...
3,3,1000,11,20,0.7,0.2,0.05,Aleatória,Inteiro,1,1,341,341,6.784231,6.180340,1.000000,b'\t\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x...
4,0,1000,11,100,0.7,0.2,0.05,Ordenada,Permutação,1,1,11,11,6.206155,6.180340,1.000000,b'\x05\x00\x00\x00\x06\x00\x00\x00\x07\x00\x00...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,3,1000,11,100,0.7,0.2,0.05,Aleatória,Inteiro,0,0,1001,1001,9.042760,7.295413,0.847154,b'\x05\x00\x00\x00\t\x00\x00\x00\x03\x00\x00\x...
3596,0,1000,11,50,0.7,0.2,0.05,Ordenada,Permutação,1,1,39,39,6.281230,6.180340,1.000000,b'\x04\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00...
3597,1,1000,11,50,0.7,0.2,0.05,Ordenada,Inteiro,1,1,12,12,6.208499,6.180340,1.000000,b'\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00...
3598,2,1000,11,50,0.7,0.2,0.05,Aleatória,Permutação,1,1,13,13,6.221852,6.180340,1.000000,b'\x06\x00\x00\x00\x07\x00\x00\x00\x04\x00\x00...


In [7]:
CalcularSR(resultados)

Topologia Aleatória            Ordenada           
Tipo        Inteiro Permutação  Inteiro Permutação
P  mu                                             
11 20          23.0      100.0    100.0      100.0
   50          66.0      100.0    100.0      100.0
   100         75.0      100.0    100.0      100.0
21 20           0.0       30.0     99.0       22.0
   50           0.0       52.0    100.0       54.0
   100          1.0       59.0    100.0       66.0
31 20           0.0        0.0     95.0        0.0
   50           0.0        7.0    100.0        1.0
   100          0.0       13.0    100.0       17.0

In [19]:
CalcularMBF(resultados)[['mean','std']].loc[31]

mean                                       std             \
Topologia  Aleatória             Ordenada            Aleatória              
Tipo         Inteiro Permutação   Inteiro Permutação   Inteiro Permutação   
mu                                                                          
20         21.750858  16.193575  7.709272  16.185660  1.698797   1.495554   
50         19.822515  13.653653  6.818386  13.983179  1.720383   1.796891   
100        18.127937  12.451868  6.590632  12.391086  1.917479   1.693458   

                                
Topologia  Ordenada             
Tipo        Inteiro Permutação  
mu                              
20         1.292896   1.200615  
50         0.327768   1.715097  
100        0.112364   1.838678

In [20]:
CalcularAES(resultados)

count                                       mean              \
Topologia Aleatória            Ordenada              Aleatória               
Tipo        Inteiro Permutação  Inteiro Permutação     Inteiro  Permutação   
P  mu                                                                        
11 20          23.0      100.0    100.0      100.0  414.565217  103.530000   
   50          66.0      100.0    100.0      100.0  241.515152   36.040000   
   100         75.0      100.0    100.0      100.0  149.520000   18.850000   
21 20           NaN       34.0     99.0       31.0         NaN  638.000000   
   50           NaN       52.0    100.0       54.0         NaN  341.211538   
   100          1.0       59.0    100.0       66.0  408.000000  209.033898   
31 20           NaN        NaN     95.0        NaN         NaN         NaN   
   50           NaN        9.0    100.0        3.0         NaN  712.222222   
   100          NaN       14.0    100.0       18.0         NaN  667.428571   

                                          std              ...      50%  \
Topologia    Ordenada               Aleatória              ... Ordenada   
Tipo          Inteiro  Permutação     Inteiro  Permutação  ...  Inteiro   
P  mu                                                      ...            
11 20       18.860000  105.160000  245.999598   68.582982  ...     12.0   
   50        5.760000   36.920000  282.919029   22.038039  ...      4.0   
   100       3.020000   19.000000  211.371158   12.113416  ...      3.0   
21 20       86.111111  622.548387         NaN  152.271090  ...     70.0   
   50       36.530000  324.981481         NaN  173.312179  ...     25.5   
   100      22.180000  216.924242         NaN  153.430313  ...     18.0   
31 20      221.905263         NaN         NaN         NaN  ...    176.0   
   50       94.500000  794.000000         NaN  205.220478  ...     77.5   
   100      54.850000  588.555556         NaN  185.100270  ...     46.0   

                           75%                                      max  \
Topologia            Aleatória            Ordenada            Aleatória   
Tipo      Permutação   Inteiro Permutação  Inteiro Permutação   Inteiro   
P  mu                                                                     
11 20           89.0     566.0     133.75    25.00     134.75     943.0   
   50           29.0     335.0      50.50     7.25      52.00     989.0   
   100          16.5     251.0      21.25     5.00      22.25     814.0   
21 20          622.0       NaN     748.25   106.50     820.00       NaN   
   50          277.0       NaN     425.50    47.25     396.75       NaN   
   100         156.0     408.0     230.50    26.00     230.00     408.0   
31 20            NaN       NaN        NaN   286.50        NaN       NaN   
   50          797.0       NaN     802.00   115.25     806.50       NaN   
   100         552.5       NaN     784.00    70.00     713.25       NaN   

                                          
Topologia            Ordenada             
Tipo      Permutação  Inteiro Permutação  
P  mu                                     
11 20          297.0    124.0      625.0  
   50          102.0     28.0      240.0  
   100          98.0      9.0       46.0  
21 20          934.0    648.0      987.0  
   50          787.0    197.0      954.0  
   100         798.0     91.0      907.0  
31 20            NaN    765.0        NaN  
   50          998.0    429.0      816.0  
   100         898.0    159.0      903.0  

[9 rows x 32 columns]